# Micro Credit Defaulter Model

## Data Description
A Microfinance Institution (MFI) is an organization that offers financial services to low income populations.Build a model which can be used to predict in terms of a probability for each loan transaction, whether the customer will be paying back the loaned amount within 5 days of insurance of loan.

Import required libraries

In [ ]:
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

## Loading the dataset

In [ ]:
df=pd.read_csv(r"C:\Users\jothu\Downloads\Data file.csv")

df.head()

Data has been imported and we can see the first 5 rows of the data using df.head() function

## Exploratory Data Analysis

In [ ]:
df.shape

This is the shape of data

In [ ]:
pd.set_option('display.max_rows',None)

In [ ]:
df.dtypes

These are the datatypes of each columns which includes integer,object and float values

In [ ]:
df.columns

These are the columns of data

In [ ]:
df.info()

There are 21 float values 13 integer values and 3 object values

In [ ]:
df.isnull().sum()

There are no null values

In [ ]:
df.describe(include='all')

Mean is higher than median in aon,daily_decr90,rental30,rental90,last_rech_date_ma.

There is a huge difference between 75th percentile and max coulumn in aon,daily_decr30,daily_decr90,rental90,cnt_loans90.

Standard deviation is higher in aon,daily_decr90,maxamnt_loans30.

In [ ]:
df.label.unique()

This column has two values hence target data is binary

In [ ]:
df.msisdn.unique()

In [ ]:
df.aon.unique()

In [ ]:
df.rental30.unique()

In [ ]:
df.maxamnt_loans30.unique()

In [ ]:
df.pcircle.unique()

This column has only one value so we can delete it because its not useful

In [ ]:
df.drop('pcircle',axis=1,inplace=True)

In [ ]:
df.drop('msisdn',axis=1,inplace=True)

Extracting year from date

In [ ]:
df["pdate"]=pd.to_datetime(df["pdate"])
df["pdate"]

In [ ]:
df["pdate"]=df["pdate"].dt.month

In [ ]:
df["pdate"]

In [ ]:
df.dtypes

All the object data are converted

In [ ]:
df.tail()

In [ ]:
sns.heatmap(df.isnull())

There are some values missing or null present in data

In [ ]:
dfcor=df.corr()
dfcor

In [ ]:
plt.figure(figsize=[22,12])
sns.heatmap(dfcor,annot=True)
plt.title("Correlation Matrix")
plt.show()

daily_decr30 and daily_decr90 are positively correlated with each other.

rental30 and rental90 are positively correlated to each other.

cnt_loans30 and fr_ma_rech90 are negatively correlated with each other.

Label is positively correlated to all the columns

In [ ]:
df['label'].plot.hist()

1 has highest frequency

In [ ]:
plt.scatter(df['daily_decr30'],df['daily_decr90'])
plt.show()

This shows the relation between daily_decr90 and daily_decr30

In [ ]:
plt.scatter(df['rental30'],df['rental90'])
plt.show()

Scatter plot stating the relation between rental30 and rental90

In [ ]:
df['fr_ma_rech90'].plot.hist()

0 to 15 has the highest frequency

## Checking Outliers using boxplot

In [ ]:
df.iloc[:,0:15].boxplot(figsize=[20,8])
plt.subplots_adjust(bottom=0.25)
plt.show()

There are outliers in aon,daily_decr30,daily_decr90,rental30 and few other columns

In [ ]:
df.iloc[:,15:30].boxplot(figsize=[20,8])
plt.subplots_adjust(bottom=0.25)
plt.show()

There are outliers in many columns

In [ ]:
df.iloc[:,30:40].boxplot(figsize=[20,8])
plt.subplots_adjust(bottom=0.25)
plt.show()

There are ouliers in amnt_loans90, payback30, payback90

## Removing Outliers

In [ ]:
from scipy.stats import zscore
z=np.abs(zscore(df.drop(['label','maxamnt_loans30','pdate'],axis=1)))
z

In [ ]:
threshold=3
print(np.where(z>3))

In [ ]:
df_new=df[(z<3).all(axis=1)]

In [ ]:
 df.shape

In [ ]:
df_new.shape

In [ ]:
loss_percent=(209593-162204)/209593*100
print(loss_percent)

## Checking skewness

In [ ]:
x=df_new.drop("label",axis=1)

In [ ]:
x.skew()

There some skewness in many columns

## Checking Skewness

In [ ]:
from sklearn.preprocessing import power_transform
df_new_credit=power_transform(x)
df_new_credit=pd.DataFrame(df_new_credit,columns=x.columns)

In [ ]:
x=df_new_credit

In [ ]:
x.tail()

In [ ]:
y=df_new["label"]

In [ ]:
y.head()

In [ ]:
## from imblearn.over_sampling import SMOTE
 
## x_train_over, y_train_over = SMOTE(k_neighbors=3).fit_resample(x, y)

In [ ]:
## x_train_over

In [ ]:
## from imblearn.under_sampling import NearMiss

## x_train_miss, y_train_miss= NearMiss().fit_resample(x_train, y_train.ravel())

In [ ]:
## x_train_miss

## Finding the best Random_State

In [ ]:
from sklearn.linear_model import LogisticRegression
maxAccu=0 
maxRS=0
for i in range(1,200):
    x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.30,random_state=i)
    LR=LogisticRegression()
    LR.fit(x_train,y_train)
    predrf=LR.predict(x_test)
    acc=accuracy_score(y_test,predrf)
    if acc>maxAccu:
        maxAccu=acc
        maxRS=i
print("Best Score is",maxAccu," on Random_state",maxRS)

## Creating train-test split

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.30,random_state=70)

In [ ]:
model=[GaussianNB(), SVC(), DecisionTreeClassifier(), KNeighborsClassifier(), LogisticRegression(), RandomForestClassifier()]

In [ ]:
for m in model: 
    m.fit(x_train,y_train)
    m.score(x_train,y_train)
    predm=m.predict(x_test) 
    print('Accuracy_score of',m,'is :')
    print(accuracy_score(y_test,predm))
    print(confusion_matrix(y_test,predm))
    print(classification_report(y_test,predm))
    print('\n')

RandomForest classifier  has highest accuracy compared to others which is 91

## Cross validation

In [ ]:
svr=SVC()

In [ ]:
lr=LogisticRegression()
dt=DecisionTreeClassifier()
rf=RandomForestClassifier()
knn=KNeighborsClassifier()
gb=GaussianNB()

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
gbscores = cross_val_score(gb,x,y,cv=5)
print(gbscores.mean())

In [ ]:
svrscores = cross_val_score(svr,x,y,cv=5) 
print(svrscores.mean()) 

In [ ]:
dtscores = cross_val_score(dt,x,y,cv=5)
print(dtscores.mean())

In [ ]:
knnscores = cross_val_score(knn,x,y,cv=5)
print(knnscores.mean())

In [ ]:
lrscores = cross_val_score(lr,x,y,cv=5)
print(lrscores.mean())

In [ ]:
rfscores = cross_val_score(rf,x,y,cv=5)
print(rfscores.mean()) 

Cross validation scores can be observed

Minimum difference between accuracy_score and cross validation for Support Vector Machine is less that is 1, so SVC will be the best model.

## Hyper parameter Tunning

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
SVC().get_params().keys()

In [ ]:
parameters= {'max_iter':np.arange(2,15),
            'kernel':['linear','poly']}

In [ ]:
GCV=GridSearchCV(SVC(),parameters,cv=5)

In [ ]:
GCV.fit(x_train,y_train)

In [ ]:
GCV.best_params_

In [ ]:
GCV_pred=GCV.best_estimator_.predict(x_test)

In [ ]:
accuracy_score(y_test,GCV_pred)

## AUC_ROC Curve

In [ ]:
from sklearn.metrics import roc_curve,auc
fpr,tpr,thresholds= roc_curve(predm,y_test)
roc_auc=auc(fpr,tpr)

plt.figure()   
plt.plot(fpr,tpr,color='darkorange',lw=10, label='ROC curve (area= %0.2f)'% roc_auc)
plt.plot([0,1], [0,1], color='navy', lw=10, linestyle='--')
plt.xlim([0.0,1.0])
plt.ylim([0.0,1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Charateristics')
plt.legend(loc="lower right" )
plt.show()

## Saving the best Model

In [ ]:
import joblib
joblib.dump(GCV.best_estimator_,"Micro Credit Defaulter.pkl")

In [ ]:
mod=joblib.load('Micro Credit Defaulter.pkl')